In [ ]:
# import wbgapi as wb
# import pandas as pd
# from tqdm import tqdm

# # Get the entire indicator dataset
# indicators = wb.series.list()
# indicator_ids = [indicator['id'] for indicator in indicators]
# print(len(indicator_ids))


1496


In [ ]:
# import pandas as pd

# df = pd.DataFrame()

# pbar = tqdm(total=len(indicator_ids))
# for indicator in indicator_ids:
#     try: 
#         data = wb.data.DataFrame(indicator, mrv=1)
#         df = pd.concat([df, data])
#     except:
#         print("Unable to fetch data for indicator: ", indicator)
#     pbar.update(1)
# pbar.close()


 12%|█▏        | 180/1496 [00:49<05:35,  3.93it/s]

Unable to fetch data for indicator:  DT.NFL.UNID.CD


 37%|███▋      | 556/1496 [04:12<10:32,  1.49it/s]  /home/jcdutoit/miniconda3/envs/acme/lib/python3.12/site-packages/wbgapi/data.py:343: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3083842126000000000000' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[index_key, column_key] = np.nan if row['value'] is None else row['value']
100%|██████████| 1496/1496 [24:50<00:00,  1.00it/s]


In [ ]:
# df['MS.MIL.XPND.CN'] = df['MS.MIL.XPND.CN'].astype(float)
# df.to_parquet('data.parquet', engine='pyarrow', compression='snappy')


In [ ]:
# print(df.columns)
# print(df.index)
# df['ISO_A3'] = df.index

Index(['AG.CON.FERT.PT.ZS', 'AG.CON.FERT.ZS', 'AG.LND.AGRI.K2',
       'AG.LND.AGRI.ZS', 'AG.LND.ARBL.HA', 'AG.LND.ARBL.HA.PC',
       'AG.LND.ARBL.ZS', 'AG.LND.CREL.HA', 'AG.LND.CROP.ZS',
       'AG.LND.EL5M.RU.K2',
       ...
       'VA.PER.RNK.LOWER', 'VA.PER.RNK.UPPER', 'VA.STD.ERR', 'VC.BTL.DETH',
       'VC.IDP.NWCV', 'VC.IDP.NWDS', 'VC.IDP.TOCV', 'VC.IHR.PSRC.FE.P5',
       'VC.IHR.PSRC.MA.P5', 'VC.IHR.PSRC.P5'],
      dtype='object', length=1495)
Index(['ABW', 'AFE', 'AFG', 'AFW', 'AGO', 'ALB', 'AND', 'ARB', 'ARE', 'ARG',
       ...
       'VIR', 'VNM', 'VUT', 'WLD', 'WSM', 'XKX', 'YEM', 'ZAF', 'ZMB', 'ZWE'],
      dtype='object', name='economy', length=397670)


/tmp/ipykernel_47663/484940997.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['ISO_A3'] = df.index


In [1]:
from DataPipeline import DataPipeline
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

class GeoPlotter():

    def __init__(self, df=None):
        # if df is None:
        #     self.df = DataPipeline().clean(["name_change"])
        if 'ISO_A3' not in df.columns:
            raise ValueError("This dataframe does not have an ISO_A3 column. Please use the DataPipeline change_name transform to prepare it for plotting")
        else: 
            self.df = df
        world = gpd.read_file('worldmap.gpkg')
        self.merged = world.merge(self.df, left_on='ISO_A3', right_on='ISO_A3')


    def plot(self, col_name, year, continent='all', country='all'):
        fig, ax = plt.subplots(1, figsize=(10, 4))
        df = self.merged[self.merged['Year'].astype(str) == year]
        df.plot(column=col_name, ax=ax, legend=True)
        ax.set_title(col_name)
        plt.show()
    
    def animate(self, col_name, video_name):
        fig, ax = plt.subplots(1, figsize=(10, 7))

        def update(year):
            df = self.merged[self.merged['Year'].astype(str) == str(year)]
            df.plot(column=col_name, ax=ax)
            ax.set_title(col_name + "\n" + str(year))

        animation.writer = animation.writers['ffmpeg']
        ani = animation.FuncAnimation(fig, update, 
            frames=sorted(set(self.df['Year'])),
            interval=self.df['Year'].max() - self.df['Year'].min() / 10)
        ani.save(video_name)

In [2]:
df = pd.read_parquet('data.parquet', engine='pyarrow')
gp = GeoPlotter(df)
gp.plot(col_name='MS.MIL.XPND.CN')


NameError: name 'pd' is not defined

In [14]:
import wbgapi as wb
wb.db = 2

for i in wb.topic.list():
    print(i)


{'id': '1', 'value': 'Agriculture & Rural Development', 'sourceNote': "For the 70 percent of the world's poor who live in rural areas, agriculture is the main source of income and employment. But depletion and degradation of land and water pose serious challenges to producing enough food and other agricultural products to sustain livelihoods here and meet the needs of urban populations. Data presented here include measures of agricultural inputs, outputs, and productivity compiled by the UN's Food and Agriculture Organization."}
{'id': '2', 'value': 'Aid Effectiveness', 'sourceNote': 'Aid effectiveness is the impact that aid has in reducing poverty and inequality, increasing growth, building capacity, and accelerating achievement of the Millennium Development Goals set by the international community. Indicators here cover aid received as well as progress in reducing poverty and improving education, health, and other measures of human welfare.'}
{'id': '3', 'value': 'Economy & Growth', 